In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, pipeline
from datasets import load_from_disk
from tqdm import tqdm
from vllm import LLM, SamplingParams
import numpy as np
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

import os

/home/ma-user/work/code_dev/miniconda3/envs/sft/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_completion(prompts, model, tokenizer=None, max_tokens=4096, temperature=0, top_p=0.95):
    # stop_token_ids = [151329, 151336, 151338]
    # 创建采样参数。temperature 控制生成文本的多样性，top_p 控制核心采样的概率
    sampling_params = SamplingParams(temperature=temperature, top_p=top_p,  max_tokens=max_tokens)
    # 初始化 vLLM 推理引擎
    llm = LLM(model=model, tokenizer=tokenizer, trust_remote_code=True, tensor_parallel_size=1, max_seq_len_to_capture=8192)
    outputs = llm.generate(prompts, sampling_params, use_tqdm=True)
    return outputs

In [3]:
from datasets import load_from_disk
processed_dataset = load_from_disk("/home/ubuntu/test_disk/health_prompt")
processed_dataset

Dataset({
    features: ['prompt'],
    num_rows: 14322
})

In [4]:
processed_dataset = load_from_disk("/home/ubuntu/test_disk/health_prompt")

# filter_dataset = load_from_disk("/root/wildchat/my_list_dataset")

# flat_list = list(itertools.chain(*[x if isinstance(x, list) else [x] for x in tqdm(processed_dataset["content"])]))
# data_list = list(itertools.chain(*[x if isinstance(x, list) else [x] for x in tqdm(filter_dataset["content"])]))


prompts = processed_dataset["prompt"][:10000]
# print(prompts[0])
model = "/home/ubuntu/test_disk/qwen7b"
outputs = get_completion(prompts, model, temperature=1, top_p=0.9)

INFO 12-18 15:18:24 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
INFO 12-18 15:18:24 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='/home/ubuntu/test_disk/qwen7b', speculative_config=None, tokenizer='/home/ubuntu/test_disk/qwen7b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=/home/ubuntu/test_di

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  2.15it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.22it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  2.16it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.12it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.14it/s]



INFO 12-18 15:18:27 model_runner.py:1077] Loading model weights took 14.2487 GB
INFO 12-18 15:18:30 worker.py:232] Memory profiling results: total_gpu_memory=23.65GiB initial_memory_usage=14.75GiB peak_torch_memory=18.60GiB memory_usage_post_profile=14.77GiB non_torch_memory=0.51GiB kv_cache_size=2.17GiB gpu_memory_utilization=0.90
INFO 12-18 15:18:30 gpu_executor.py:113] # GPU blocks: 2544, # CPU blocks: 4681
INFO 12-18 15:18:30 gpu_executor.py:117] Maximum concurrency for 32768 tokens per request: 1.24x
INFO 12-18 15:18:33 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-18 15:18:33 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INF

Processed prompts:   0%|          | 0/10000 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

WARNING 12-18 15:19:19 scheduler.py:1481] Sequence group 48 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


Processed prompts: 100%|██████████| 10000/10000 [25:10<00:00,  6.62it/s, est. speed input: 6199.68 toks/s, output: 576.93 toks/s]


In [70]:
from tqdm import tqdm
output_list = []
for i, output in enumerate(tqdm(outputs)):
    output_list.append(output.outputs[0].text)
output_list[:5]

100%|██████████| 10000/10000 [00:00<00:00, 159468.93it/s]


['{\n    "question1": "What factors might contribute to Spain leading in international immunization rankings for COVID-19 vaccination rates compared to neighboring countries?",\n    "question2": "How does the organizational structure of a country\'s vaccination campaign potentially influence its immunization rate, and what examples from the text support this idea?"\n}',
 '{\n    "question1": "How does the narrative explore the relationship between pharmacy regulations and the pharmaceutical industry\'s marketing strategies for erectile dysfunction medications, and what ethical concerns arise from these practices?",\n    "question2": "In the context of the text, how do different characters\' actions and decisions reflect the broader themes of consent and manipulation in personal relationships, and what potential consequences can arise from these dynamics?"\n}',
 '{\n    "question1": "What are the key advantages of the new non-invasive technique developed by the University of Kent for ea

In [7]:
# import json
# from datasets import Dataset
# # 去除前缀
# cleaned_data = [item.strip("```json\n") for item in output_list]
# # print(cleaned_data)
# # 替换所有单引号为双引号，确保 JSON 格式合法
# # cleaned_data = [item.replace("'", '"') for item in cleaned_data]
# # cleaned_data = cleaned_data[:9]
# cleaned_data

In [29]:
import json

# 保存为 JSON 文件
with open("/home/ubuntu/test_disk/ins_gen_output_list/ins_gen_output_list.json", "w") as file:
    json.dump(output_list, file)

In [63]:
import json

# 打开并读取 JSON 文件
with open("/home/ubuntu/test_disk/ins_gen_output_list/ins_gen_output_list.json", "r", encoding="utf-8") as file:
    data = json.load(file)  # 解析文件中的 JSON 数据
    print(data[:5])  

['{\n    "question1": "What factors might contribute to Spain leading in international immunization rankings for COVID-19 vaccination rates compared to neighboring countries?",\n    "question2": "How does the organizational structure of a country\'s vaccination campaign potentially influence its immunization rate, and what examples from the text support this idea?"\n}', '{\n    "question1": "How does the narrative explore the relationship between pharmacy regulations and the pharmaceutical industry\'s marketing strategies for erectile dysfunction medications, and what ethical concerns arise from these practices?",\n    "question2": "In the context of the text, how do different characters\' actions and decisions reflect the broader themes of consent and manipulation in personal relationships, and what potential consequences can arise from these dynamics?"\n}', '{\n    "question1": "What are the key advantages of the new non-invasive technique developed by the University of Kent for earl

In [73]:
from datasets import Dataset

# 初始化存储成功解析的 JSON 数据和成功序号的列表
data = []
successful_indexes = []

# 使用列表推导解析 JSON，跳过有错误的项
for idx, item in enumerate(output_list):
    try:
        # 尝试解析 JSON
        parsed_item = json.loads(item)
        data.append(parsed_item)
        successful_indexes.append(idx)  # 保存成功解析的序号
    except json.JSONDecodeError:
        # 如果解析失败，打印错误信息并跳过当前项
        # print(f"第 {idx} 项解析失败: {item}")
        continue
# 使用 datasets 库创建数据集
dataset = Dataset.from_list(data)
# 输出数据集内容
print(dataset)
len(successful_indexes)

Dataset({
    features: ['question1', 'question2'],
    num_rows: 9948
})


9948

In [75]:
dataset[0], successful_indexes[0]

({'question1': 'What factors might contribute to Spain leading in international immunization rankings for COVID-19 vaccination rates compared to neighboring countries?',
  'question2': "How does the organizational structure of a country's vaccination campaign potentially influence its immunization rate, and what examples from the text support this idea?"},
 0)

In [9]:
question1 = dataset["question1"]
question1_dict = {"question": question1}
question1_set = Dataset.from_dict(question1_dict)
print(question1_set)

question2 = dataset["question2"]
question2_dict = {"question": question2}
question2_set = Dataset.from_dict(question2_dict)
print(question2_set)

question1_set.save_to_disk("/home/ubuntu/test_disk/ins1")
question2_set.save_to_disk("/home/ubuntu/test_disk/ins2")

Dataset({
    features: ['question'],
    num_rows: 100
})
Dataset({
    features: ['question'],
    num_rows: 100
})


Saving the dataset (1/1 shards): 100%|██████████| 100/100 [00:00<00:00, 23242.29 examples/s]


In [76]:
np.save('/home/ubuntu/test_disk/successful_indexes.npy', successful_indexes)